Use [RISE](https://github.com/damianavila/RISE) to display the slides.

```bash
pip install RISE
```

<H1 style="text-align: center;">
EstNLTK õpituba
</H1>

<H4 style="text-align: center;">
Dage Särg, Sven Laur
</H4>

<H4 style="text-align: center;">
Keeletehnoloogia konverents 5.11.2019
</H4>

<H4 style="text-align: center;">
Materjalid zip-failina: TODO <br> 
Materjalid githubis: https://tinyurl.com/estnltk-github
</H4>

## 0. Installimine
EstNLTK-d on kõige parem installida läbi Anaconda, mille saab alla laadida siit https://www.anaconda.com/distribution/ (Python 3.7 versioon).

Seejärel avage terminaliaken ning looge ja aktiveerige conda keskkond, mis kasutab Python 3.6:
```bash
conda create -n estnltk python=3.6 -y
conda activate estnltk
```
Installige loodud keskkonda EstNLTK 1.6 ning jupyter:
```bash
conda install -c estnltk -c conda-forge estnltk
conda install jupyter
```
Liikuge käsureal kausta, kuhu laadisite alla õpitoa materjalid ning käivitage notebook:
```bash
jupyter notebook EstNLTK
```

### Olulisemad käsud, milleta Notebookis läbi ei saa:
**Shift+Enter**: jooksuta lahter ja liigu järgmise juurde

### Shortcutid command mode'is (st kursorit pole lahtris!) 

**a**: uus lahter üles (_above_)

**b**: uus lahter alla (_below_)

**dd**: kustuta lahter (_delete_)

## Plaan:
1. Teksti segmenteerimine
2. Morfoloogiline analüüs ja ühestamine
3. Standardmärgendajad e taggerid
4. Oma märgendajate kirjutamine
5. Tekstikollektsioonide töötlemine

## 1. Teksti segmenteerimine

In [ ]:
from estnltk import Text
# Teksti töötlemiseks peame tegema stringi Text objektiks
text = Text("Festivalil osales üle 30 000 muusikahuvilise.")
text

In [ ]:
# tag_layer() meetod märgib peale standardsed analüüsikihid,
# mida on vaja pea kõigi keeletöötlusülesannete juures
text.tag_layer()

In [ ]:
text.text

In [ ]:
# Tokens e sõned - mitte alati lingvistiliselt motiveeritud
text.tokens

In [ ]:
# Words e sõnad - mõned tokenid ühendatakse edasiseks töötluseks kokku
text.words

In [ ]:
# Laused - punkt ei toimi alati eraldajana
text = Text('Tartu Rattaralli toimub 29. mail 2020. \
Tartu Rattaralli stardi- ja finišipaik \
on traditsiooniliselt Tartu kesklinnas, Turu tänaval.')
text.tag_layer()
text.sentences

In [ ]:
# Tekst koosneb lausetest, mis koosnevad omakorda sõnadest
for sentence in text.sentences:
    print(' Lause: ', sentence.enclosing_text)
    for word in sentence:
        # Väljastame sõna ja sõnaliigi
        print( word.text, word.morph_analysis.partofspeech[0] )
    print()

In [ ]:
# On võimalik märgendada ka osalauseid
text = Text('Nendel, kes minu ja Oudekki kaotusele loodavad, \
on ettekujutus, et rahval polegi hääli.')

text.tag_layer(['clauses'])
text.clauses

## 2. Morfoloogiline analüüs & ühestamine

<img src="oad.png">

In [ ]:
# Morf analüüs eesti keele puhul baassamm 
t = Text('Mida ubadest teha? Oad võib salatisse panna.').tag_layer()
t.morph_analysis

In [ ]:
# Võime vaadata ka vaid parajasti huvitavaid atribuute, mitte kogu analüüsi
t.morph_analysis['text', 'lemma', 'partofspeech']

### Morfoloogiline ühestamine

<img src="k2rbes.png">

In [ ]:
# Ühestajal ei ole alati õigus - vahel tahaks kustutatud variante ka
t = Text('Mees oli kärbes.').tag_layer()
t.morph_analysis

In [ ]:
# Morfanalüsaatori parameetrite muutmiseks peame kasutama resolverit
from estnltk.resolve_layer_dag import make_resolver

resolver = make_resolver(
                 disambiguate=False, # keelame ühestamise
                 guess=True,
                 propername=True,
                 phonetic=False,
                 compound=True)

In [ ]:
# Morf analüüs ilma ühestamiseta - võib anda oodatust rohkem mitmesusi
text = Text("Mees oli kärbes.")
text.tag_layer(resolver=resolver)['morph_analysis']

#### Näide: leiame kõik tekstis olevad nimisõnad

In [ ]:
my_text = Text('Nagu nimigi reedab, on nurgasaag kõige \
tõhusam tööriist erinevate puitdetailide lõikamiseks, \
kus eesmärgiks on saavutada täpne lõikenurk ning oluline on \
lõikenurga seadistamise võimalus. Näiteks pildiraamide \
meisterdamisel, kus on oluline, et detailide lõikenurgad \
oleksid kõik täpselt 45 kraadi. Sellisel juhul on nurgasaag \
täiuslikuks tööriistaks, sest tagab täpsuse ja lõike korratavuse. \
Üldiselt on valdav osa nurgasaage seadistatavad 45-kraadise \
lõikenurga alla vähemalt ühes suunas. Lisaks võimaldavad mõned \
saed veel ka saetera kaldenurga seadistamist, mis tuleb kasuks \
keerukamate detailide lõikamisel. Nurgasaag on väga tõhus ka \
kitsamate, kuni 30 cm laiuste puulaudade või muude puitdetailide \
ristlõigete tegemiseks ehk järkamiseks, mida tuleb palju ette näiteks \
puitkonstruktsioonide ehitamisel või ka näiteks terrassilaudade \
või puitparketi paigaldamisel.')
my_text.tag_layer()

In [ ]:
noun_lemmas = []
for lemmas, postags in zip(my_text.lemma, my_text.partofspeech):
    if 'S' in postags: # text.lemma ja partofspeech on listid, kuna analüüse võib olla mitu
        noun_lemmas += lemmas
noun_lemmas  

from collections import Counter
Counter(noun_lemmas).most_common()

#### Näide: leiame kõik infinitiivset verbi sisaldavad laused:

In [ ]:
infinitive_sentences = []
for sent in my_text.sentences: # vaatame teksti lause kaupa
    for form in sent.form: # vaatame läbi kõik lause sõnade vormiinfod
        if 'da' in form:
            a = sent.enclosing_text # lause tekst stringina
            infinitive_sentences.append(a)
            break
infinitive_sentences

## 3. Märgendajad e taggerid
#### võimaldavad vajadusel lisada kihte, mida läheb vaja mingis konkreetses analüüsitöövoos

### Ajaväljendite tuvastamine

In [ ]:
text3 = Text('EKA sisearhitektuuri osakond ja RMK \
avavad neljapäeval kell 16.00 \
RMK Tallinna kontoris (Toompuiestee 24) näituse')

In [ ]:
# Märgendame ajaväljendid
from estnltk.taggers import TimexTagger

tagger = TimexTagger()
text3.tag_layer()
tagger.tag( text3 )

In [ ]:
text3.timexes

In [ ]:
# Soovi korral võime määratleda teksti loomise aja
text3 = Text('EKA sisearhitektuuri osakond ja RMK \
avavad neljapäeval kell 16.00 \
RMK Tallinna kontoris (Toompuiestee 24) näituse').tag_layer()
text3.meta['document_creation_time'] = '2019-10-27'
tagger.tag(text3)

In [ ]:
text3.timexes

### Aadresside tuvastamine

In [ ]:
# Toimub kahes etapis
from estnltk.taggers import AddressPartTagger, AddressGrammarTagger
address_token_tagger = AddressPartTagger(output_layer='address_tokens')
address_tagger = AddressGrammarTagger(output_layer='addresses', 
                                      input_layer='address_tokens')

In [ ]:
text = Text("Ootame teid 2. novembril külla \
aadressil Aia 6, Tartu.").tag_layer(['words'])

In [ ]:
# Esiteks märgime peale võimalikud aadresside komponendid
address_token_tagger.tag(text)["address_tokens"]

In [ ]:
# Teiseks leiame aadressid sealt, kus sobivad komponendid järjest esinevad
address_tagger.tag(text)['addresses']

## 4. Oma märgendajate kirjutamine

### PhraseTagger
#### võimaldab märgendada kihis järjest esinevaid elemente mingi atribuudi alusel

Proovime kirjutada taggerit, mis märgendaks lihtsaid nimisõnafraase.

In [ ]:
from estnltk.taggers import PhraseTagger

# Kasutame fraaside esmaseks määratlemiseks sõnaliike
phrase_list = [
               { '_phrase_': ('A', 'S')},
               { '_phrase_':  ('C', 'S')},
               { '_phrase_':  ('U', 'S')}
              ]

In [ ]:
# Defineerime taggeri, mis phrase_list muutujas olevaid fraasitüüpe märgendaks
phrase_tagger = PhraseTagger(output_layer='noun_phrases',
                      input_layer='morph_analysis',
                      input_attribute='partofspeech',
                      vocabulary=phrase_list,
                      key='_phrase_')

In [ ]:
# Rakendame kirjutatud taggerit morfanalüüsitud tekstile
t = Text('Viimasedki pardid lendasid soojemale maale, \
kui jää läks liiga paksuks jõest toidu hankimiseks.').tag_layer()
phrase_tagger.tag(t)

In [ ]:
# Leiame nimisõnafraasid
# Puuduvad algvormid
# Paremate tulemuste jaoks peaks arvesse võtma rohkem kui sõnaliike
t.noun_phrases

In [ ]:
# Dekoraator võimaldab lisada oma uuele kihile atribuute - lisame lemmad
def decorator(span, annotation):
    annotation['lemmas'] = ' '.join([l[0] for l in span.lemma])
    return True

In [ ]:
# Uus phrase_tagger, mis paneb uude kihti ka fraasid algvormis
phrase_tagger2 = PhraseTagger(output_layer='noun_phrases2',
                      input_layer='morph_analysis',
                      input_attribute='partofspeech',
                      vocabulary=phrase_list,
                      key='_phrase_',
                      output_attributes = ['lemmas'],
                      decorator = decorator)

In [ ]:
# Olemas ilusad algvormis fraasid
# Vaja oleks ka sõna vormiinfot arvestada
phrase_tagger2.tag(t)
t.noun_phrases2

In [ ]:
# Täiendame dekoraatorit - arvestame ka vormide ühilduvust
def decorator2(span, annotation):
    annotation['lemmas'] = ' '.join([l[0] for l in span.lemma])
    
    ninataga_sg = ['sg ter', 'sg ab', 'sg kom', 'sg es']
    ninataga_pl = ['pl ter', 'pl ab', 'pl kom', 'pl es']
    # Omadussõna ja nimisõna samas vormis -> OK
    if span[0].form == span[1].form:
        return True
    # Omadussõna ainsuse omastavas ja nimisõna 4 viimases käändes ainsuses -> OK
    elif span[0].form[0] == 'sg g' and span[1].form[0] in ninataga_sg:
        return True
    # Omadussõna mitm omastavas ja nimisõna 4 viimases käändes mitm -> OK
    elif span[0].form[0] == 'pl g' and span[1].form[0] in ninataga_pl:
        return True
    # Kõik muud juhud -> ei sobi fraas
    else: 
        return False

In [ ]:
phrase_tagger3 = PhraseTagger(output_layer='noun_phrases3',
                      input_layer='morph_analysis',
                      input_attribute='partofspeech',
                      vocabulary=phrase_list,
                      key='_phrase_',
                      output_attributes = ['lemmas'],
                      decorator = decorator2)

In [ ]:
# Märgendame kolmanda nimisõnafraaside kihi
phrase_tagger3.tag(t)

In [ ]:
# Tulemused vastavad ootustele
 t.noun_phrases3

In [ ]:
# Fraase on lihtne stringidena kätte saada
for i in t.noun_phrases3:
    print(i.lemmas)

#### Näide: leiame, millised nimisõnafraasid esinevad kõige sagedamini eesti vanasõnades

In [ ]:
with open("data/proverbs.txt", "r", encoding = 'utf8') as fin:
    # Failis on iga vanasõna eraldi real - saame listi vanasõnadest
    proverbs = fin.readlines()

In [ ]:
from collections import Counter
from tqdm import tqdm

# Loendur fraaside kokkulugemiseks
noun_phrases_counts = Counter()

for text in tqdm(proverbs): # vaatame vanasõnade listi järjest läbi
    t = Text(text).tag_layer() # teeme vanasõna Text objektiks ja analüüsime
    phrase_tagger3.tag(t) # märgime peale nimisõnafraasid oma parima taggeriga
    if len(t.noun_phrases3) > 0: 
        for p in t.noun_phrases3: # suurendame loendurit vastava fraasi kohal
            noun_phrases_counts[p.lemmas] += 1

In [ ]:
# Saamegi kätte sagedasemad nimisõnafraasid
noun_phrases_counts.most_common(5)